# U. .U ワンちゃんネコちゃん判定 ^. .^
### MNISTのモデルを使って写真から犬か猫を判定しよう！（学習が失敗している例）

### Keras
ディープラーニングのライブラリ　バックエンドでTensorFlowを使用（デフォルト、Theanoなどに変更もできる）
- 公式リファレンス https://keras.io/ja/
- バックエンドの変更 https://qiita.com/samacoba/items/3e29f0ac61c2f369892b

#### ライブラリのバージョン
- python 3.7.0 ※3.8.1も動作確認済み
- keras 2.3.1
- numpy 1.17.3
- opencv 4.1.1　※pipの場合はopencvのみでOKです
- matplotlib 3.1.2

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

### 画像を読み込む
- TRAIN_DIR：学習データの場所
- TEST_DIR：テストデータの場所
- IMG_WIDTH：画像をリサイズする幅
- IMG_HEIGHT：画像をリサイズする高さ

In [ ]:
TRAIN_DIR = './cat-and-dog/training_set/training_set'
TEST_DIR = './cat-and-dog/test_set/test_set'
# MNISTと同じサイズ
IMG_WIDTH = 28
IMG_HEIGHT = 28

#### 学習データ読み込み

In [ ]:
# 学習データをリストにする
train_data = []
train_label = []

# ねこ画像4000枚（training_set/cats/cat.1.jpg～cat.4000.jpg）
for i in range(1,4001):
    image_name = 'cat.' + str(i) + '.jpg'
    # opencvで画像を読み込む
    image = cv2.imread(TRAIN_DIR + '/cats/' + image_name)
    # opencvで画像を100×100にリサイズ
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    # リサイズした画像を学習データリストに追加
    train_data.append(image)
    # 2クラス分類（犬or猫）なのでラベルは0と1を使用
    train_label.append(0)
    
# いぬ画像4000枚（training_set/dogs/dog.1.jpg～dog.4000.jpg）
for i in range(1,4001):
    image_name = 'dog.' + str(i) + '.jpg'
    image = cv2.imread(TRAIN_DIR + '/dogs/' + image_name)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    train_data.append(image)
    train_label.append(1)

# 学習画像を表示 かなり粗いです。。。
plt.imshow(train_data[7904])
plt.show()

In [ ]:
# 幅,高さ,色情報
train_data[0].shape

###### 学習データをグレースケールにする

In [ ]:
# 色情報を削除する
train_data_gray = []
for i in range(0, 8000):
    train_data_gray.append(cv2.cvtColor(train_data[i], cv2.COLOR_BGR2GRAY))

plt.imshow(train_data_gray[7904])
plt.show()

In [ ]:
# 色情報がなくなっている
train_data_gray[0].shape

In [ ]:
# 画像と正解ラベルはリスト型なので配列に変換する
train_data_gray = np.array(train_data_gray)
train_label = np.array(train_label)

# 画像の行と列を掛け算して一次元配列にする（28×28なので784）
train_data_gray = train_data_gray.reshape(8000, 784)

In [ ]:
# 正解ラベル（cat,dog）をone-hotエンコード
train_label = keras.utils.to_categorical(train_label, 2)

#### テストデータ読み込み

In [ ]:
# テストデータをリストにする
test_data = []
test_label = []

# ねこ画像1000枚（training_set/cats/cat.4001.jpg～cat.5000.jpg）
for i in range(4001,5001):
    image_name = 'cat.' + str(i) + '.jpg'
    image = cv2.imread(TEST_DIR + '/cats/' + image_name)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    test_data.append(image)
    test_label.append(0)
    
# いぬ画像1000枚（training_set/dogs/dog.4001.jpg～dog.5000.jpg）
for i in range(4001,5001):
    image_name = 'dog.' + str(i) + '.jpg'
    image = cv2.imread(TEST_DIR + '/dogs/' + image_name)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    test_data.append(image)
    test_label.append(1)

# グレースケールにする
test_data_gray = []
for i in range(0, 2000):
    test_data_gray.append(cv2.cvtColor(test_data[i], cv2.COLOR_BGR2GRAY))
    
# ニューラルネットワークに画像データを入れるためにリスト→配列に変換
test_data_gray = np.array(test_data_gray)

# 画像の行と列を掛け算して一次元配列にする（28×28なので784）
test_data_gray = test_data_gray.reshape(2000, 784)

# 正解ラベル（cat,dog）をone-hotエンコード
test_label = keras.utils.to_categorical(test_label, 2)

### ディープラーニングモデル構築
MNIST（手書き数字認識）と同じモデルを構築します

In [ ]:
model = Sequential()

# ノード512個、活性化関数ReLU、入力値は一次配列で784要素（28×28）
model.add(Dense(512, activation='relu', input_shape=(784,)))

# 20％の入力ノードを無効化
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

# cat,dogの2クラス分類
model.add(Dense(2, activation='sigmoid'))

# モデルの精度を評価する方法を設定
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# モデルの構成を表示
model.summary()

### 学習・テスト実行
先ほど構築したCNNモデルに学習データ、テストデータを投入して犬or猫の判定ができているかをチェックする<br>
**※処理に時間が掛かるので注意**
- バッチサイズ：4000枚の学習画像を何枚ずつに分けて学習させるか（外れ値の影響を少なくする）
- エポック数：学習する回数（4000枚/バッチサイズしたn個のサブセットが全部学習して1エポック）

**犬or猫の判定で正解率が50%くらいになります（学習できていない）**

In [ ]:
# 学習条件もMNISTと同じにしている
batch_size = 128
epochs = 20

# 学習画像、学習ラベル、バッチサイズ、エポック数、進捗を表示、テストデータ
history = model.fit(x=train_data_gray,
                    y=train_label, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1,
                    validation_data=(test_data_gray,test_label))

### 学習・テスト結果をグラフにして表示
- 青線：学習データの正解率
- 赤線：テストデータの正解率

In [ ]:
# グラフのサイズ
plt.figure(figsize=(10,6))

# 青線：学習結果
plt.plot(history.history['accuracy'], color='b')
# 赤線：テスト結果
plt.plot(history.history['val_accuracy'], color='r')

plt.tick_params(labelsize=18)
plt.title('epoch - accuracy graph')
# 縦軸：正解率（モデルの精度）
plt.ylabel('accuracy')
# 横軸：エポック数
plt.xlabel('epoch')
plt.legend(['train','test'])
plt.show()